Ver si existe relacion entre nivel de satisfaccion  en el trabajo y la rotacion de los empleados y ver la magnitud de la relacion  

H0 "Existe una relación entre el nivel de satisfacción en el trabajo y la rotación de empleados en la empresa. Se sospecha que los empleados con niveles de satisfacción más bajos tienen una mayor probabilidad de dejar la empresa".  

Para ello deberéis crear una columna nueva en base al nivel de satisfacción en el trabajo para crear dos grupos, las condiciones para crear los grupos son:

    - Grupo A (Control): Empleados con un nivel de satisfacción en el trabajo igual o superior a 3 en una escala de 1 a 5

    - Grupo B (Variante): Empleados con un nivel de satisfacción en el trabajo inferior a 3 en la misma escala.  
    
La métrica principal que debes usar es la tasa de rotación de empleados (Attrition) en cada grupo. Recordemos que, en esta columna, "No" indica que el empleado no ha dejado la empresa (sin rotación), mientras que "Yes" indica que el empleado ha dejado la empresa (con rotación).


- Analiza los resultados.

- Calcula la magnitud de esta relación utilizando estadísticas como la diferencia de medias por ejemplo.  




In [73]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Realizar los distintos test
# ------------------------------------------------------------------------------

from scipy.stats import ttest_ind, norm, chi2_contingency, f_oneway
from sklearn.linear_model import LinearRegression
import scipy.stats as stats
from scipy import stats
from scipy.stats import levene
from scipy.stats import shapiro, poisson, chisquare, expon, kstest

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore") 

In [3]:
df = pd.read_csv("Files/df_fase1.csv", index_col=0)
df.head(2)

,IdEmployee,EmployeeNumber,Age,DateBirth,Attrition,BusinessTravel,DailyRate,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,RemoteWork
0,0,"162,0",52,1972,No,no-data,684.0,6,3,no-data,1,Male,51.0,3,5,Research Director,3,no-data,6462.0,7,No,13,3,3,0,no-data,5,3,20,15,15,True
1,1,"259,0",53,1971,No,no-data,699.0,1,4,Life Sciences,1,Male,65.0,2,5,Manager,3,no-data,5678.0,0,no-data,14,3,1,1,"34,0",5,3,33,11,9,True


- Divide a los empleados en los grupos A y B según los criterios establecidos.

In [ ]:
# División del grupo A, empleados con un nivel de satisfacción igual o superior a 3.
# Vamos a imputar una columna, que nos indique, por fila, a qué grupo pertenece según su JobSatisfaction.

df['GroupSatisfaction'] = df['JobSatisfaction'].apply(lambda x: 'A' if x >= 3 else 'B')

In [8]:
df.sample(3)

,IdEmployee,EmployeeNumber,Age,DateBirth,Attrition,BusinessTravel,DailyRate,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,RemoteWork,GroupSatisfaction
249,249,"734,0",42,1982,No,no-data,314.0,1,3,no-data,1,Male,59.0,2,5,Manager,3,no-data,19562.0,1,No,12,3,2,1,"22,0",3,3,22,2,10,False,A
97,97,"1167,0",43,1981,Yes,travel_frequently,481.0,15,3,Life Sciences,1,Male,44.0,3,4,Sales Executive,1,Single,2447.0,0,no-data,12,3,2,0,no-data,2,2,21,13,14,True,B
328,328,"1745,0",31,1993,No,travel_frequently,1312.0,2,4,no-data,1,Female,78.0,2,1,Research Scientist,1,no-data,26427.0,0,no-data,16,3,4,0,"10,0",2,3,9,0,7,False,B


- Calcula la tasa de rotación (porcentaje de empleados que dejaron la empresa) en cada grupo.

In [15]:
# rotacion = Los que han dejado la empresa / el total de los empleados registrados

# Attrition --> los que tienen un YES. Estos son los que HAN DEJADO LA EMPRESA. 

rotationA = (len(df[(df['Attrition'] == 'Yes') & (df['GroupSatisfaction'] == 'A')])) / len(df['GroupSatisfaction'] == 'A') * 100
rotationA

8.461025982678215

In [23]:
print(len(df[(df['GroupSatisfaction'] == 'A')]))
print(rotationA)

917
8.461025982678215


Del total de 927 que tienen una satisfacción de 3 o más, el 8.461 % NO están en la empresa ya.

In [26]:
rotationB = (len(df[(df['Attrition'] == 'Yes') & (df['GroupSatisfaction'] == 'B')])) / len(df['GroupSatisfaction'] == 'B') * 100
rotationB

7.59493670886076

In [27]:
print(len(df[(df['GroupSatisfaction'] == 'B')]))
print(rotationB)

584
7.59493670886076


Del total de 584 que tienen una satisfacción de menos de 3 puntos, el 7.5949 % ya no están en la empresa.

Vamos a mostrar en un gráfico los dos grupos y cómo se distribuyen:

In [48]:
# Debemos meterlos en un DataFrame primeramente para poderlos representar:

rotation_data = pd.DataFrame({
    'Group':['A', 'B'],
    'Rotation Rate':[rotationA, rotationB]
})

In [ ]:
#### FALTA PONER EL GRAFIQUITO ESTE ASQUEROSO QUE NO ME SALE :(
### PARA VER EN COLORINCHOS CUÁNTO COGE DE ROTACION EN CADA GRUPO

sns.barplot(x=rotationA, data=groupA, index=[0])

plt.xlabel('Groups')
plt.ylabel('Rotation Rate %')
plt.title('Tasa de Rotación por Grupos')

# Ajustar el diseño
plt.tight_layout()

# Mostrar el gráfico
plt.show()

- Realiza un análisis estadístico para determinar si hay una diferencia significativa en la tasa de rotación entre los grupos A y B.

In [56]:
# Generamos cada grupo:

groupA = df[df['GroupSatisfaction'] == 'A']
groupA.sample(2)

,IdEmployee,EmployeeNumber,Age,DateBirth,Attrition,BusinessTravel,DailyRate,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,RemoteWork,GroupSatisfaction
1406,1302,no-data,60,1964,No,travel_rarely,818.0,6,2,Medical,1,Male,58.0,3,1,Human Resources,3,Married,25657.0,8,no-data,17,3,4,0,no-data,2,2,2,2,2,False,A
1558,1454,no-data,43,1981,No,travel_rarely,1059.0,9,2,no-data,1,Male,93.0,2,5,Manager,4,no-data,26362.0,8,No,22,4,4,0,"24,0",2,3,1,0,1,False,A


In [57]:
groupB = df[df['GroupSatisfaction'] == 'B']
groupB.sample(2)

,IdEmployee,EmployeeNumber,Age,DateBirth,Attrition,BusinessTravel,DailyRate,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,RemoteWork,GroupSatisfaction
700,700,"819,0",34,1990,Yes,travel_rarely,118.0,16,3,Marketing,1,Female,69.0,3,2,Sales Executive,1,Single,26507.0,5,no-data,15,3,3,0,no-data,3,3,3,0,2,False,B
973,973,"816,0",30,1994,Yes,travel_rarely,805.0,1,2,Life Sciences,1,Female,36.0,2,1,Laboratory Technician,1,Married,6689.0,1,no-data,11,3,4,1,"1,0",1,3,1,0,0,True,B


In [ ]:
# Primero comprobamos si cada conjunto de datos sigue una distribución normal:



#### Prueba de Shapiro-Wilk

La prueba de Shapiro-Wilk es una prueba estadística paramétrica que se utiliza para evaluar la normalidad de una muestra. La hipótesis nula (H0) de esta prueba es que la muestra proviene de una población con distribución normal.

Hipótesis nula (H0): Los datos se distribuyen normalmente.
Hipótesis alternativa (H1): Los datos no se distribuyen normalmente.

##### Interpretación de los resultados
p-value > alpha: No hay suficiente evidencia para rechazar la hipótesis nula. Se concluye que los datos se ajustan a una distribución normal.
p-value <= alpha: Se rechaza la hipótesis nula. Los datos no se ajustan a una distribución normal.

In [79]:
# Primero comprobamos qué tipo de distribución tenemos con la prueba de Shapiro:

p_value = shapiro(df['IdEmployee']).pvalue

alpha = 0.05

if p_value > alpha:
    print("Los datos se ajustan a una distribución normal (p-value =", p_value, ")")
else:
    print("Los datos no se ajustan a una distribución normal (p-value =", p_value, ")")

Los datos no se ajustan a una distribución normal (p-value = 3.997729973138971e-21 )


In [ ]:
# Haremos la prueba Z porque conocemos la desviación estándar de la población y es apropiada para muestras grandes:



In [ ]:
#'Atrittion'  para rotación
# Sobre la satisfaccion hay:'JobSatisfaction'